In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import Capital Bike Share DataFrames
df = pd.read_csv('last_week.csv')

In [4]:
# Checking the no. of rows and columns in the given DataFrames
df.shape

(57934, 14)

In [5]:
# Checking empty cells in the columns for DataFrames
df.isnull().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name    4991
start_station_id      4991
end_station_name      5422
end_station_id        5422
start_lat                0
start_lng                0
end_lat                 86
end_lng                 86
member_casual            0
day                      0
dtype: int64

In [6]:
# Droping empty rows from the DataFrame
df= df.dropna()

In [8]:
# Top 5 (by default) for DataFrame with capacity information
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,day
0,B3FA23181ADC06FE,docked_bike,2021-06-25 00:51:53,2021-06-25 01:51:40,Georgia Ave & Spring St,32035.0,New Hampshire & Kirklynn Ave,32061.0,38.999388,-77.031555,38.988101,-76.988422,casual,2021-06-25
1,6A153F2AD6700EE1,classic_bike,2021-06-22 06:28:23,2021-06-22 06:33:19,Merrifield Cinema & Merrifield Town Center,32235.0,Merrilee Dr & Lauren Elizabeth Lane,32234.0,38.870093,-77.229970,38.879355,-77.230681,casual,2021-06-22
2,7265E4C759F1EDC0,classic_bike,2021-06-22 22:27:28,2021-06-22 22:30:39,Columbia & Ontario Rd NW,31296.0,15th & Harvard St NW,31103.0,38.924088,-77.040787,38.926124,-77.035421,member,2021-06-22
3,18FB278BE5F2B0C7,docked_bike,2021-06-26 20:49:57,2021-06-26 22:44:29,1st & H St NW,31638.0,1st & H St NW,31638.0,38.900358,-77.012108,38.900358,-77.012108,casual,2021-06-26
4,294E78FE02E1C65F,classic_bike,2021-06-22 20:35:41,2021-06-22 20:45:27,5th & K St NW,31600.0,17th St & Massachusetts Ave NW,31267.0,38.903040,-77.019027,38.908142,-77.038359,member,2021-06-22


In [14]:
# Checking Data Type for the given DataFrames
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id      float64
end_station_name       object
end_station_id        float64
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
day                    object
dtype: object

In [13]:
# Checking Data Type for the given DataFrames
type(df['started_at'][0])

str

In [15]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [16]:
type(df['started_at'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [22]:
df.index = df['started_at']
df.index

DatetimeIndex(['2021-06-25 00:51:53', '2021-06-22 06:28:23',
               '2021-06-22 22:27:28', '2021-06-26 20:49:57',
               '2021-06-22 20:35:41', '2021-06-26 10:46:29',
               '2021-06-21 21:36:19', '2021-06-22 20:03:32',
               '2021-06-26 12:56:49', '2021-06-21 16:33:50',
               ...
               '2021-06-26 17:15:27', '2021-06-24 09:56:27',
               '2021-06-26 14:58:17', '2021-06-23 21:02:32',
               '2021-06-25 12:28:37', '2021-06-23 11:48:10',
               '2021-06-25 00:07:51', '2021-06-25 17:44:53',
               '2021-06-25 17:43:45', '2021-06-21 20:00:18'],
              dtype='datetime64[ns]', name='started_at', length=50625, freq=None)

In [25]:
df.loc['2021-06-25']

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,day
started_at,,,,,,,,,,,,,,
2021-06-25 00:51:53,B3FA23181ADC06FE,docked_bike,2021-06-25 00:51:53,2021-06-25 01:51:40,Georgia Ave & Spring St,32035.0,New Hampshire & Kirklynn Ave,32061.0,38.999388,-77.031555,38.988101,-76.988422,casual,2021-06-25
2021-06-25 08:42:40,6768B11274C9150A,electric_bike,2021-06-25 08:42:40,2021-06-25 08:57:22,Walter Reed Dr & 8th St S,31096.0,Court House Metro / 15th St & N Uhle St,31089.0,38.866102,-77.087815,38.890544,-77.085031,member,2021-06-25
2021-06-25 19:19:35,4AABDB29F5BF91B2,classic_bike,2021-06-25 19:19:35,2021-06-25 19:32:12,N Veitch St & 20th St N,31029.0,Roosevelt Island,31062.0,38.896923,-77.086502,38.896553,-77.067140,member,2021-06-25
2021-06-25 22:41:19,2661C0C2B136D09C,classic_bike,2021-06-25 22:41:19,2021-06-25 22:46:24,Army Navy Dr & S Joyce St,31006.0,18th St & S Eads St,31001.0,38.863314,-77.063416,38.857250,-77.053320,member,2021-06-25
2021-06-25 17:40:27,5246C261C72117D6,classic_bike,2021-06-25 17:40:27,2021-06-25 17:46:24,Pershing Dr & Wayne St,31065.0,Court House Metro / 15th St & N Uhle St,31089.0,38.880705,-77.085960,38.890612,-77.084801,member,2021-06-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25 00:04:10,D68EBD960F4B8C86,classic_bike,2021-06-25 00:04:10,2021-06-25 00:40:41,Independence Ave & L'Enfant Plaza SW/DOE,31633.0,17th St & Independence Ave SW,31290.0,38.887312,-77.025762,38.888097,-77.038325,casual,2021-06-25
2021-06-25 12:28:37,AD0C3E7F3BC2EDD6,docked_bike,2021-06-25 12:28:37,2021-06-25 15:34:37,17th St & Independence Ave SW,31290.0,17th St & Independence Ave SW,31290.0,38.888097,-77.038325,38.888097,-77.038325,casual,2021-06-25
2021-06-25 00:07:51,C027BB4E110C06B7,docked_bike,2021-06-25 00:07:51,2021-06-25 00:40:00,17th St & Independence Ave SW,31290.0,17th St & Independence Ave SW,31290.0,38.888097,-77.038325,38.888097,-77.038325,casual,2021-06-25


In [27]:
df.loc['2021-06-22' : '2021-06-23'].groupby('start_station_name').count()['rideable_type'].sort_values(ascending=False)

start_station_name
15th & P St NW                     60
New Hampshire Ave & T St NW        59
17th & Corcoran St NW              58
Thomas Circle                      58
Columbus Circle / Union Station    57
                                   ..
Pleasant St & MLK Ave SE            1
East West Hwy & 16th St             1
Pennsylvania & Minnesota Ave SE     1
Oxon Hill Park & Ride               1
Yuma St & Tenley Circle NW          1
Name: rideable_type, Length: 499, dtype: int64